<a href="https://colab.research.google.com/github/kwb425/class-2025-spring/blob/main/class-2025-spring_0530-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
!pip install --no-deps unsloth
!pip install datasets

In [ ]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import standardize_sharegpt
from datasets import Dataset
from transformers import TextStreamer
import json
import pandas as pd
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


# Data


* raw
> In the latest AP dispatch from Frankfurt, which details how the European Union's chief trade negotiator said he had “good calls” with Trump administration officials on Monday, David McHugh noted that the EU "has offered Trump a 'zero for zero' deal in which tariffs would be removed on industrial goods including automobiles, but the U.S. administration has said it will not lower tariffs below a 10% baseline imposed on almost all its trading partners. Trump has also announced tariffs of 25% on steel and automobiles."
That raises the question of where the negotiations, which got back on the rails over the weekend after a call between President Trump and EU Commission President Ursula von der Leyen, will go from here — and what Trump will have to say next time he discusses the $1.8 trillion trade relationship.

* pretrain
	-	one record:
		> In the latest AP dispatch from Frankfurt, which details how the European Union's chief trade negotiator said he had “good calls” with Trump administration officials on Monday, David McHugh noted that the EU "has offered Trump a 'zero for zero' deal in which tariffs would be removed on industrial goods including automobiles, but the U.S. administration has said it will not lower tariffs below a 10% baseline imposed on almost all its trading partners. Trump has also announced tariffs of 25% on steel and automobiles."
		That raises the question of where the negotiations, which got back on the rails over the weekend after a call between President Trump and EU Commission President Ursula von der Leyen, will go from here — and what Trump will have to say next time he discusses the $1.8 trillion trade relationship.

* instructional - alpaca
```
[
	{
		"Instruction": "Task we want the model to perform. 11111",
		"Input": "Optional, but useful, it will essentially be the user's query. 11111",
		"Output": "The expected result of the task and the output of the model 11111."
	},
	{
		"Instruction": "Task we want the model to perform. 22222",
		"Input": "Optional, but useful, it will essentially be the user's query. 22222",
		"Output": "The expected result of the task and the output of the model. 22222"
	},
]
```

* instructional - full fine-tuning | Lora fine-tuning
	- one record:
		```
		"""Below is an instruction that describes a task, paired with an input that provides further context.
		Write a response that appropriately completes the request.

		### Instruction:
		Task we want the model to perform. 11111

		### Input:
		Optional, but useful, it will essentially be the user's query. 11111

		### Response:
		The expected result of the task and the output of the model. 11111"""
		```
	- one record:
		```
		"""Below is an instruction that describes a task, paired with an input that provides further context.
		Write a response that appropriately completes the request.

		### Instruction:
		Task we want the model to perform. 22222

		### Input:
		Optional, but useful, it will essentially be the user's query. 22222

		### Response:
		The expected result of the task and the output of the model. 22222"""
		```

* conversational - shareGPT
```
{
	"conversations": [
		{
			"from": "system",
			"value": "You are a helpful assistant."
		},
		{
			"from": "human",
			"value": "Can you help me make pasta carbonara?"
		},
		{
			"from": "gpt",
			"value": "Would you like the traditional Roman recipe, or a simpler version?"
		},
		{
			"from": "human",
			"value": "The traditional version please"
		},
		{
			"from": "gpt",
			"value": "The authentic Roman carbonara uses just a few ingredients: pasta, guanciale, eggs, Pecorino Romano, and black pepper. Would you like the detailed recipe?"
		}
	]
}
```

* conversational (shareGPT): full fine-tuning | Lora fine-tuning
	- one record:
		```
		<|im_start|>system
		You are a helpful assistant.<|im_end|>
		<|im_start|>user
		Can you help me make pasta carbonara?<|im_end|>
		<|im_start|>assistant
		<think>
		</think>
		Would you like the traditional Roman recipe, or a simpler version?<|im_end|>
		<|im_start|>user
		The traditional version please<|im_end|>
		<|im_start|>assistant
		<think>
		</think>
		The authentic Roman carbonara uses just a few ingredients: pasta, guanciale, eggs, Pecorino Romano, and black pepper. Would you like the detailed recipe?<|im_end|>
		```

* conversational - ChatML
```
{
	"messages": [
		{
			"role": "system",
			"content": "You are a helpful assistant."
		},
		{
			"role": "user",
			"content": "What is 1+1?"
		},
		{
			"role": "assistant",
			"content": "It's 2!"
		},
		{
			"role": "user",
			"content": "What is 2+2?"
		},
		{
			"role": "assistant",
			"content": "It's 4!"
		}
	]
}
```

* conversational (ChatML): full fine-tuning | Lora fine-tuning
	- one record:
		```
		<|im_start|>system
		You are a helpful assistant.<|im_end|>
		<|im_start|>user
		What is 1+1?<|im_end|>
		<|im_start|>assistant
		<think>
		</think>
		It's 2!<|im_end|>
		<|im_start|>user
		What is 2+2?<|im_end|>
		<|im_start|>assistant
		<think>
		</think>
		It's 4!<|im_end|>
		```

In [ ]:
with open('class-2025-spring_0530-0.json', 'r') as f:
    loaded_data = json.load(f)

dataset = Dataset.from_list(loaded_data)

## Unsloth

In [ ]:
# https://huggingface.co/unsloth
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-14B",
    max_seq_length = 2048,   # Context length - can be longer, but uses more memory
    load_in_4bit = True,     # 4bit uses much less memory
    full_finetuning = False,
)

==((====))==  Unsloth 2025.5.9: Fast Qwen3 patching. Transformers: 4.52.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/168k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.59G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/4.67k [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 32,
    use_gradient_checkpointing = "unsloth",
)

Unsloth 2025.5.9 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


# Data Check


In [ ]:
organized_dataset = standardize_sharegpt(dataset)
processed_texts = tokenizer.apply_chat_template(
    organized_dataset["conversations"],
    tokenize = False,
)
df = pd.DataFrame({"text": processed_texts})
processed_dataset = Dataset.from_pandas(df)
processed_dataset = processed_dataset.shuffle(seed = 3407)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

# Training


In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = processed_dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        max_steps = 30,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to="none"
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100 | Num Epochs = 3 | Total steps = 30
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 128,450,560/14,000,000,000 (0.92% trained)


Step,Training Loss
1,3.108700
2,3.224100
3,3.128100
4,2.350100
5,2.358900
6,2.008500
7,1.617800
8,1.338900
9,1.193500
10,0.943800


# Inference

According to the `Qwen-3` team, the recommended settings for reasoning inference are `temperature = 0.6, top_p = 0.95, top_k = 20`
For normal chat based inference, `temperature = 0.7, top_p = 0.8, top_k = 20`

In [ ]:
messages = [
    {"role" : "user", "content" : "자유심증주의란 무엇인가요? 어떻게 적용되나요? 헷갈리는데?"},
    # {"role" : "user", "content" : "경복궁이 뭐에요?"}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = False, # Disable thinking
)
output = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 256, # Increase for longer outputs!
    temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

법관이 자신의 양심과 논리에 따라 증거를 평가하는 원칙입니다.<|im_end|>


# Saving & loading

In [ ]:
model.save_pretrained("my_lora_model")
tokenizer.save_pretrained("my_lora_model")

('my_lora_model/tokenizer_config.json',
 'my_lora_model/special_tokens_map.json',
 'my_lora_model/chat_template.jinja',
 'my_lora_model/vocab.json',
 'my_lora_model/merges.txt',
 'my_lora_model/added_tokens.json',
 'my_lora_model/tokenizer.json')

Loading trained model

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "my_lora_model",
    max_seq_length = 2048,
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.5.9: Fast Qwen3 patching. Transformers: 4.52.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 